In [1]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import geoplotlib

TRACE_DATA = './car-bus-simulation/output/events_lat_long.csv'
COLUMNS = ['time', 'action', 'vid', 'lat', 'lon']

# Define input size number of rows to be read from the csv into the pandas dataframe
UNITY=1
VID='4858_52'#'6311_1'
SMALL=1000
MEDIUM=5000000
LARGE=None
INPUT_SIZE=SMALL

mdf = pd.read_csv(TRACE_DATA, nrows=INPUT_SIZE, names=COLUMNS, delimiter=";", header=None)
       
# All data points from a given vehicle id
def filter_by_id(dataframe, id):
    return dataframe.loc[dataframe['vid'] == id]

# The first time a vehicle id registered an action in the simulation
def origin(dataframe,id):
    return filter_by_id(dataframe, id).iloc[[0]]
        
# The last time a vehicle id registered an action in the simulation
def destiny(dataframe, id):
    return filter_by_id(dataframe, id).iloc[[-1]]

tool_tip = lambda x: 'time: ' + str(x["time"]) + ', lat: ' + str(x["lat"]) + ', lon: ' + str(x["lon"])


In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import heapq 
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import dateutil
import datetime 
import sys
sys.path.append("../")

from historical.readData.estimateData3 import read

from historical.readData.estimateData3 import read
from historical.readData.estimateData3 import search_travels
from historical.readData.estimateData3 import estimate
from historical.readData.estimateData3 import stops_distance

from historical.readData.travels3 import haversine2

def calcula_dist_shape(selec_linhas):
    distance_all_shapes = {}
    # print 'Calculando distancias dos shapes'
    '''calcula todas as distancias dos shapes'''
    
    pth_files_GTFS = "../historical/readData/dados/gtfs/"        
        
    trips = pd.read_csv(pth_files_GTFS + 'trips.txt', sep=',')
    shapes = pd.read_csv(pth_files_GTFS + 'shapes.txt', sep=',')
    
    for l, trip_id in enumerate(selec_linhas):

        trip = trips[trips.trip_id == trip_id]
        trip_shape = shapes[shapes['shape_id'].isin(trip['shape_id'])]
        shapelat = trip_shape.shape_pt_lat.tolist()
        shapelon = trip_shape.shape_pt_lon.tolist()

        # distancias dos shapes
        lon1 = shapelon[0]
        lat1 = shapelat[0]
        totalcal = [0.]
        dist = [0.]
        for lat2, lon2 in zip(shapelat[1:], shapelon[1:]):
            d = haversine2(lat1, lon1, lat2, lon2)
            d = d * 1000
            dist.append(d)
            totalcal.append(totalcal[-1] + d)
            lat1 = lat2
            lon1 = lon2
        distance_all_shapes[trip_id] = [shapelat, shapelon, totalcal]
    return distance_all_shapes

def stops_distance(linha):

    distances = calcula_dist_shape([linha])
    totalcal = distances[linha][2]
    
    pth_files_GTFS = "../historical/readData/dados/gtfs/"
    

    trips = pd.read_csv(pth_files_GTFS + 'trips.txt', sep=',')
    shapes = pd.read_csv(pth_files_GTFS + 'shapes.txt', sep=',')
    stops = pd.read_csv(pth_files_GTFS + 'stops.txt', sep=',')
    stopid = pd.read_csv(pth_files_GTFS + 'stop_times.txt', sep=',')   

    ida = trips[trips.trip_id == linha]
    shapeida = shapes[shapes['shape_id'].isin(ida['shape_id'])]
    idalat = shapeida.shape_pt_lat.tolist()
    idalon = shapeida.shape_pt_lon.tolist()
    total = shapeida.shape_dist_traveled.tolist()

    temp1 = stopid[stopid.trip_id == linha]
    stopsida = stops[stops['stop_id'].isin(temp1['stop_id'])]
    stopsida = stopsida.set_index('stop_id')
    stopsida = stopsida.reindex(index=temp1['stop_id'])

    stopslat = stopsida.stop_lat.tolist()
    stopslon = stopsida.stop_lon.tolist()

    dpontos = [None] * len(stopslat)
    index = 0
    lat = idalat
    lon = idalon
    total = 0
    p = ['depois'] * len(stopslat)
    for latb, lonb, i in zip(stopslat, stopslon, range(len(stopslat))):
        lat = lat[index:]
        lon = lon[index:]
        nn = haversine2(latb, lonb, np.array(lat), np.array(lon)) * 1000
        index = nn.argmin()
        total = total + index
        if index == 0:
            dpontos[i] = nn[index]
#            descontardist = nn[index]
        else:
            if totalcal[total] >= totalcal[total - 1] + nn[index - 1]:
                p[i] = 'antes'
            dpontos[i] = totalcal[total - 1] + nn[index - 1]

    mid_points = []
#    mid_points.append(200)
    p1 = dpontos[0]
    for p2 in dpontos[1:]:
        mid_points.append(p1 + (p2-p1)/2)
        p1 = p2
    dpontos = [i/1000. for i in dpontos]
    mid_points = [i/1000. for i in mid_points]
    # dpontos = [val-descontardist for val in dpontos]
    # stopsida.reset_index(drop=True, inplace=True)
    # stopsida['distancia'] = dpontos
    # stopsida.sort_values('distancia',axis=0,inplace=True)
    # stopsida.reset_index(drop=True, inplace=True)

    # gmap = gmplot.GoogleMapPlotter(idalat[0], idalon[0], 19)
    # gmap.marker(idalat[0],idalon[0], title='Test text')
    # gmap.plot(idalat, idalon, 'blue')
    # gmap.scatter(idalat, idalon, c='blue', size=10, marker=True, titles=totalcal)
    # gmap.scatter(stopsida['stop_lat'].tolist(), stopsida['stop_lon'].tolist(), c='red', size=10, marker=True,
    #             titles=["Ponto " + str(i + 1) + ": dist: " + str(stopsida['distancia'][i]) + ' ' + p[i] for i in
    #                     range(len(dpontos))])

    # gmap.draw('pontos.html')

    return dpontos, mid_points

def read(filedata,filerep):
    df0 = pd.read_pickle(filedata, compression=None)
    with open(filerep, 'rb') as handle:
        reps = pickle.load(handle, encoding='latin1')
    return df0, reps

selected_line = ['8700-10-1']

pth_files_GTFS = "../historical/readData/dados/gtfs/"

trips = pd.read_csv(pth_files_GTFS + 'trips.txt', sep=',')
shapes = pd.read_csv(pth_files_GTFS + 'shapes.txt', sep=',')
stops = pd.read_csv(pth_files_GTFS + 'stops.txt', sep=',')
stopid = pd.read_csv(pth_files_GTFS + 'stop_times.txt', sep=',')   

periods = ['morning', 'm_peak', 'i_peak', 'a_peak', 'night']

line = selected_line[0]
pth_files_lines = "../historical/readData/"
p, mp = stops_distance(line)
df, reps = read(pth_files_lines + "trips_" + line + ".dsk", pth_files_lines + "interps_" + line + ".rep")   



In [8]:
# Custom layer
from geoplotlib.layers import BaseLayer
from geoplotlib.layers import HotspotManager
from geoplotlib.utils import BoundingBox
from geoplotlib.core import BatchPainter
import random
import time

# trajectories = df_temporal
trajectories = mdf

class DotDensityLayer(BaseLayer):

    def __init__(self, data, color=None, point_size=2, f_tooltip=None):
        """Create a dot density map
        :param data: data access object
        :param color: color
        :param point_size: point size
        :param f_tooltip: function to return a tooltip string for a point
        """
        self.frame_counter = 23
        self.data = data
        self.last = self.data.drop_duplicates(subset=['vid'], keep='first')
        self.color = color
        if self.color is None:
            self.color = [255,0,0]
        self.point_size = point_size
        self.f_tooltip = f_tooltip

        self.hotspots = HotspotManager()


    def invalidate(self, proj):
        self.last = self.data.loc[self.data['time'] == self.frame_counter]
        lon_at = self.last['lon']
        lat_at = self.last['lat']
        
        self.x, self.y = proj.lonlat_to_screen(lon_at, lat_at)
        


    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter = BatchPainter()
        current = self.data.loc[self.data['time'] == self.frame_counter]
        
        self.last = pd.concat([self.last, current]).drop_duplicates(subset='vid', keep='last')
        lon_at = self.last['lon']
        lat_at = self.last['lat']
        
        x, y = proj.lonlat_to_screen(lon_at, lat_at)
        
        if self.f_tooltip:
            for i in range(0, len(x)):
                record = {k: self.data[k][i] for k in self.data.keys()}
                self.hotspots.add_rect(x[i] - self.point_size, y[i] - self.point_size,
                                       2*self.point_size, 2*self.point_size,
                                       self.f_tooltip(record))

        self.painter.set_color(self.color)
        self.painter.points(x, y, 2*self.point_size, False)
        
        self.painter.batch_draw()
        picked = self.hotspots.pick(mouse_x, mouse_y)
        if picked:
            ui_manager.tooltip(picked)
        self.frame_counter += 1
        #time.sleep(0.4)


    def bbox(self):
        return BoundingBox.from_points(lons=self.data['lon'], lats=self.data['lat'])
        
geoplotlib.add_layer(DotDensityLayer(trajectories))
geoplotlib.show()

In [6]:
# input_size_to_human = "small_"
if INPUT_SIZE in range(1, SMALL + 1):
    input_size_to_human = "small"
elif INPUT_SIZE in range(SMALL, MEDIUM + 1):
    input_size_to_human = "medium"
elif INPUT_SIZE == None or INPUT_SIZE > MEDIUM:
    input_size_to_human = "large"
else:
    raise Exception("Invalid input size")
                    
output_file =  ("adds_dest_coordinates_%s.csv" % input_size_to_human)

def calculate_trajectories(force=False):
    import os.path
    if(force==False and os.path.exists(output_file)):
        print("Trajectories calculated in %s, use force=True to recalculate" % output_file)
        return

    counts = mdf.groupby(['vid'], as_index=False)

    print("Input Size: %s" % input_size_to_human)
    print("Number of unique vehicles: %s" % len(counts.count()))

    def shift_lat_lon(df):
        df.loc[:, 'dest_lat'] = df.shift(-1).loc[:, 'lat']
        df.loc[:, 'dest_lon'] = df.shift(-1).loc[:, 'lon']
        return df

    res = counts.apply(shift_lat_lon)
    res.sort_values(['vid','time'], inplace=True)
    res.reset_index(drop=True, inplace=True)

    print("Writing to csv")
    res.to_csv(output_file, index=False)
    print("Written to %s" % output_file)
    
calculate_trajectories()

Trajectories calculated in adds_dest_coordinates_small.csv, use force=True to recalculate


In [7]:
datapoints = pd.read_csv("adds_dest_coordinates.csv", delimiter=",")
#filtered = datapoints[datapoints['dest_lat'].notnull()]

geoplotlib.kde(datapoints, bw=5, cmap='Oranges', cmap_levels=20, show_colorbar=True, alpha=170)

geoplotlib.show()

FileNotFoundError: File b'adds_dest_coordinates.csv' does not exist